In [1]:
import torch.nn as nn
import torch
import random
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
def generate_data_files():
    if os.path.exists("DataSets/Jumble/jumbled.txt"): os.remove("DataSets/Jumble/jumbled.txt")
    if os.path.exists("DataSets/Jumble/unjumbled.txt"): os.remove("DataSets/Jumble/unjumbled.txt")   
    f1 = open("DataSets/Jumble/jumbled.txt","w")
    f2 = open("DataSets/Jumble/unjumbled.txt","w")
    with open("DataSets/Jumble/source.txt","r") as f:
        for line in f:
            split_line = line.lower().strip().split()
            sentence = split_line[1:]
            f2.write(' '.join(sentence)+"\n")
            random.shuffle(sentence)
            f1.write(" ".join(sentence)+"\n")
    f1.close()
    f2.close()

In [ ]:
generate_data_files()

In [ ]:
def get_count_of_words():
    word_count_dict = {}
    with open("DataSets/Jumble/jumbled.txt","r") as f:
        for line in f:
            words = line.split()
            for word in words:
                if word in word_count_dict: word_count_dict[word] += 1
                else: word_count_dict[word] = 1
    return word_count_dict

In [ ]:
word_count_dict = get_count_of_words()
print(len(word_count_dict))

In [ ]:
min_word_count = 4
count = 0
for k,v in word_count_dict.items():
    if v > min_word_count: count += 1
print(count)

In [ ]:
def replace_less_frequent_words(filename,word_count_dict,min_word_count,replace_token="<unk>"):
    with open(filename,"r") as f:
        with open("DataSets/Jumble/processed_"+filename.split("/")[-1],"w") as f1:
            for line in f:
                words = line.strip().split()
                sentence_to_write = []
                for word in words:
                    if word_count_dict[word] > min_word_count: sentence_to_write.append(word)
                    else: sentence_to_write.append(replace_token)
                sentence_to_write.append("\n")
                f1.write(" ".join(sentence_to_write))

In [ ]:
replace_less_frequent_words(
    "DataSets/Jumble/jumbled.txt",word_count_dict,min_word_count=min_word_count,replace_token="<unk>"
)

In [ ]:
replace_less_frequent_words(
    "DataSets/Jumble/unjumbled.txt",word_count_dict,min_word_count=min_word_count,replace_token="<unk>"
)

## Classes

In [3]:
class VocabBuilder:
    def __init__(self,text_corpus,unknown_token=None,pad_token=None,sos_token=None,eos_token=None):
        '''
        text_corpus = [
            sentence_1,  # sentence_1 = "a yellow car ..."
            sentence_2
            ...
        ]
        '''
        self.text_corpus = text_corpus
        self.unknown_token = unknown_token or "<unk>"
        self.pad_token = pad_token or "<pad>"
        self.sos_token = sos_token or "<sos>"
        self.eos_token = eos_token or "<eos>"
        self.word_to_index, self.index_to_word = self.get_vocabs()
                        
    def get_vocabs(self):
        word_to_index = {}
        index_count = 0
        for sentence in self.text_corpus:
            words = sentence.split()
            for word in words:
                if word not in word_to_index:
                    word_to_index[word] = index_count
                    index_count += 1
        if not self.unknown_token in word_to_index: 
            word_to_index[self.unknown_token] = index_count
            index_count += 1
        if not self.pad_token in word_to_index: 
            word_to_index[self.pad_token] = index_count
            index_count += 1
        if not self.sos_token in word_to_index: 
            word_to_index[self.sos_token] = index_count
            index_count += 1
        if not self.eos_token in word_to_index: 
            word_to_index[self.eos_token] = index_count
            index_count += 1
        index_to_word = {v:k for k,v in word_to_index.items()}
        return word_to_index, index_to_word

In [4]:
class EncodeDecode:
    def __init__(self,word_to_index,index_to_word,pad_token,unknown_token,smallcase=True):
        self.smallcase = smallcase
        self.word_to_index = word_to_index
        self.index_to_word = index_to_word
        self.pad_token = pad_token
        self.unknown_token = unknown_token
    
    def get_encoding(self,sentence):
        '''
        sentence can be a string, or a list of words
        '''
        if isinstance(sentence,str): sentence = sentence.split(" ")
        if self.smallcase: sentence =  [word.lower() for word in sentence]
        encoded_sentence = []
        for word in sentence:
            if word in self.word_to_index: encoded_sentence.append(self.word_to_index[word])
            else: encoded_sentence.append(self.word_to_index[self.unknown_token])
        return encoded_sentence
    
    def get_decoding(self,encoded_sentence):
        '''
        encoded_sentence must be a list of vocab indices.
        Ex: encoded_sentence = [24,21,4,1,..] 
        '''
        sentence = [self.index_to_word[index] for index in encoded_sentence]
        return " ".join(sentence)

In [24]:
class UnjumbleEncoderModel(nn.Module):
    def __init__(self,vocab_size,embedding_dim,num_encoder_layers,debug):
        super().__init__()
        self.debug = debug
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.relu = nn.ReLU()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=4)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_encoder_layers)
        
    def forward(self,x):
        if self.debug: 
            print("_______________________________")
            print("\t\tEncoder\t\t")
            print("_______________________________")
        if self.debug: print("Before starting: x Shape:",x.shape)
        
        x = self.embedding(x)
        x = self.relu(x)
        if self.debug: print("Embedding, x Shape:",x.shape)
        
        x = x.permute(1,0,2)
        if self.debug: print("Reshaping for Transformer, x Shape:",x.shape)
            
        op = self.transformer_encoder(x)
        if self.debug: print("Transformer Encoder, op Shape:",op.shape)
            
        return op
    
class UnjumbleDecoderModel(nn.Module):
    def __init__(self,vocab_size,embedding_dim,num_decoder_layers,debug):
        super().__init__()
        self.debug = debug
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.relu = nn.ReLU()
        self.decoder_layer = nn.TransformerDecoderLayer(d_model=embedding_dim, nhead=4)
        self.transformer_decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=num_decoder_layers)
        self.linear = nn.Linear(in_features=embedding_dim, out_features=vocab_size)
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self,x,op_from_encoder):
        if self.debug: 
            print("_______________________________")
            print("\t\tDecoder\t\t")
            print("_______________________________")
        if self.debug: print("Before starting: x Shape:",x.shape," op_from_encoder Shape:",op_from_encoder.shape)
        
        x = self.embedding(x)
        x = self.relu(x)
        if self.debug: print("Embedding, x Shape:",x.shape)
        
        x = x.permute(1,0,2)
        if self.debug: print("Reshaping for Transformer, x Shape:",x.shape)
        
        op = self.transformer_decoder(x,op_from_encoder)
        if self.debug: print("Transformer Decoder, op Shape:",op.shape)
            
        # Resizing for Linear Layer
        op = op.reshape(-1,op.shape[2])
        if self.debug: print("Reshaping op Shape:",op.shape)
        
        linear_op = self.linear(op)
        if self.debug: print("Linear linear_op Shape:",linear_op.shape)
        
        op = self.log_softmax(linear_op)
        if self.debug: print("log_softmax op Shape:",op.shape)
            
        if self.debug:print("_______________________________\n\n")
            
        return op


### Putting it all together

In [25]:
unknown_token = "<unk>"
pad_token = "<pad>"
sos_token = "<sos>"
eos_token = "<eos>"
Xe,Xd,Y = [],[],[]
with open("DataSets/Jumble/processed_jumbled.txt","r") as f:
    for line in f:
        Xe.append(line.strip()+" " +eos_token)
with open("DataSets/Jumble/processed_unjumbled.txt","r") as f:
    for line in f:
        Xd.append(sos_token+" "+line.strip())
        Y.append(line.strip()+" " +eos_token)
print(len(Xe),len(Xd),len(Y))

40460 40460 40460


In [26]:
Xtr_e, Xval_e, Xtr_d, Xval_d, Ytr, Yval = train_test_split(Xe,Xd,Y,test_size=0.1,random_state=20)
print(len(Xtr_e), len(Xval_e), len(Xtr_d), len(Xval_d), len(Ytr), len(Yval))
encoder_vocab_builder = VocabBuilder(Xtr_e,unknown_token=unknown_token,pad_token=pad_token,sos_token=sos_token,eos_token=eos_token)
decoder_vocab_builder = VocabBuilder(Xtr_d,unknown_token=unknown_token,pad_token=pad_token,sos_token=sos_token,eos_token=eos_token)
encoder_wtoi,encoder_itow = encoder_vocab_builder.word_to_index, encoder_vocab_builder.index_to_word
decoder_wtoi,decoder_itow = decoder_vocab_builder.word_to_index, decoder_vocab_builder.index_to_word
len(encoder_itow),len(encoder_wtoi),len(decoder_wtoi),len(decoder_itow)

36414 4046 36414 4046 36414 4046


(3005, 3005, 3005, 3005)

In [27]:
device = torch.device("cuda:0")
num_encoder_layers = 3
model_encoder = UnjumbleEncoderModel(
    vocab_size=len(encoder_wtoi),embedding_dim=300,num_encoder_layers=num_encoder_layers,debug=True
).to(device)
num_decoder_layers = num_encoder_layers
model_decoder = UnjumbleDecoderModel(
    vocab_size=len(encoder_wtoi),embedding_dim=300,num_decoder_layers=num_decoder_layers,debug=True
).to(device)
loss_fn = nn.NLLLoss()
optimizer_encoder = torch.optim.Adam(model_encoder.parameters(),lr=0.003)
optimizer_decoder = torch.optim.Adam(model_decoder.parameters(),lr=0.003) 

In [28]:
data_index = 6
encoder_encode_decode = EncodeDecode(encoder_wtoi,encoder_itow,pad_token,unknown_token)
decoder_encode_decode = EncodeDecode(decoder_wtoi,decoder_itow,pad_token,unknown_token)
print(Xtr_e[data_index],encoder_encode_decode.get_encoding(Xtr_e[data_index]))
print(Xtr_d[data_index],decoder_encode_decode.get_encoding(Xtr_d[data_index]))
print(Ytr[data_index],decoder_encode_decode.get_encoding(Ytr[data_index]))

model_encoder.train()
model_decoder.train()

optimizer_encoder.zero_grad()
optimizer_decoder.zero_grad()
Xe_b = torch.tensor([encoder_encode_decode.get_encoding(Xtr_e[data_index])]).to(device)
Xd_b = torch.tensor([decoder_encode_decode.get_encoding(Xtr_d[data_index])]).to(device)
Y_b = torch.tensor([decoder_encode_decode.get_encoding(Ytr[data_index])]).to(device)
print(Xe_b.shape,Xd_b.shape,Y_b.shape)
op_from_encoder = model_encoder(Xe_b)
op = model_decoder(Xd_b,op_from_encoder)

loss = loss_fn(op,Y_b.reshape(-1))
loss.backward()
optimizer_encoder.step()
optimizer_decoder.step()

print("---------------------------------------------")

optimizer_encoder.zero_grad()
optimizer_decoder.zero_grad()
Xe_b = torch.tensor([encoder_encode_decode.get_encoding(Xtr_e[data_index+1])]).to(device)
Xd_b = torch.tensor([decoder_encode_decode.get_encoding(Xtr_d[data_index+1])]).to(device)
Y_b = torch.tensor([decoder_encode_decode.get_encoding(Ytr[data_index+1])]).to(device)
print(Xe_b.shape,Xd_b.shape,Y_b.shape)
op_from_encoder = model_encoder(Xe_b)
op = model_decoder(Xd_b,op_from_encoder)
loss = loss_fn(op,Y_b.reshape(-1))
loss.backward()
optimizer_encoder.step()
optimizer_decoder.step()

a descends . snowy man mountain a <eos> [13, 47, 8, 48, 49, 50, 13, 9]
<sos> a man descends a snowy mountain . [0, 10, 47, 48, 10, 49, 50, 9]
a man descends a snowy mountain . <eos> [10, 47, 48, 10, 49, 50, 9, 3004]
torch.Size([1, 8]) torch.Size([1, 8]) torch.Size([1, 8])
_______________________________
		Encoder		
_______________________________
Before starting: x Shape: torch.Size([1, 8])
Embedding, x Shape: torch.Size([1, 8, 300])
Reshaping for Transformer, x Shape: torch.Size([8, 1, 300])
Transformer Encoder, op Shape: torch.Size([8, 1, 300])
_______________________________
		Decoder		
_______________________________
Before starting: x Shape: torch.Size([1, 8])  op_from_encoder Shape: torch.Size([8, 1, 300])
Embedding, x Shape: torch.Size([1, 8, 300])
Reshaping for Transformer, x Shape: torch.Size([8, 1, 300])
Transformer Decoder, op Shape: torch.Size([8, 1, 300])
Reshaping op Shape: torch.Size([8, 300])
Linear linear_op Shape: torch.Size([8, 3005])
log_softmax op Shape: torch.Size

In [29]:
def predict(model_encoder,model_decoder,encoder_encode_decode,decoder_itow,decoder_wtoi):
    data_index = random.randint(0,100)
    Xe_b = torch.tensor([encoder_encode_decode.get_encoding(Xval_e[data_index])]).to(device)
    print(Xval_e[data_index],Xe_b)
    print(Xval_d[data_index])
    
    model_encoder.eval()
    model_decoder.eval()
    with torch.no_grad():
        softmax_ops = []
        op_from_encoder = model_encoder(Xe_b)
        sos_word = torch.tensor([[decoder_wtoi["<sos>"]]]).to(device)
        op = model_decoder(sos_word,op_from_encoder)
        unjumbled_sentence = []
        for i in range(25):
            predicted_word = torch.argmax(op,axis=1).tolist()
#             print("Predicted .....................",predicted_word)
            unjumbled_sentence.append(decoder_itow[predicted_word[0]])
            if predicted_word[0] == decoder_wtoi["<eos>"]: break
            op = model_decoder(torch.tensor([predicted_word]).to(device),op_from_encoder)
        print("_______________________________________")
        print(unjumbled_sentence)
        print("_______________________________________")

In [33]:
predict(model_encoder,model_decoder,encoder_encode_decode,decoder_itow,decoder_wtoi)

people . sit benches on <eos> tensor([[203,   8, 401, 512,  53,   9]], device='cuda:0')
<sos> people sit on benches .
_______________________________________
['<eos>']
_______________________________________


## For Actual Training

In [34]:
device = torch.device("cuda:0")
num_encoder_layers = 3
model_encoder = UnjumbleEncoderModel(
    vocab_size=len(encoder_wtoi),embedding_dim=400,num_encoder_layers=num_encoder_layers,debug=False
).to(device)
num_decoder_layers = num_encoder_layers
model_decoder = UnjumbleDecoderModel(
    vocab_size=len(encoder_wtoi),embedding_dim=400,num_decoder_layers=num_decoder_layers,debug=False
).to(device)

loss_fn = nn.NLLLoss()
optimizer_encoder = torch.optim.Adam(model_encoder.parameters(),lr=0.0003)
optimizer_decoder = torch.optim.Adam(model_decoder.parameters(),lr=0.0003)

In [32]:
epochs = 10
for i in range(epochs):
    model_encoder.train()
    model_decoder.train()
    epoch_loss = 0
    for j in range(len(Xtr_e)):
        optimizer_encoder.zero_grad()
        optimizer_decoder.zero_grad()
        Xe_b = torch.tensor([encoder_encode_decode.get_encoding(Xtr_e[j])]).to(device)
        Xd_b = torch.tensor([decoder_encode_decode.get_encoding(Xtr_d[j])]).to(device)
        Y_b = torch.tensor([decoder_encode_decode.get_encoding(Ytr[j])]).to(device)
        op_from_encoder = model_encoder(Xe_b)
        op = model_decoder(Xd_b,op_from_encoder)
        loss = loss_fn(op,Y_b.reshape(-1))
        loss.backward()
        optimizer_encoder.step()
        optimizer_decoder.step()
        batch_loss = loss.item()
        epoch_loss += batch_loss
        if j%2000 == 0: print("Epoch:",i,"Batch:",j,"Loss:",batch_loss)
    print("______________________________________")
    print("Epoch Loss:",epoch_loss)
    predict(model_encoder,model_decoder,encoder_encode_decode,decoder_itow,decoder_wtoi)
    print("_______________________________________")

Epoch: 0 Batch: 0 Loss: 8.253918647766113
Epoch: 0 Batch: 2000 Loss: 4.433011531829834
Epoch: 0 Batch: 4000 Loss: 1.855195164680481
Epoch: 0 Batch: 6000 Loss: 4.546597003936768
Epoch: 0 Batch: 8000 Loss: 4.006786346435547
Epoch: 0 Batch: 10000 Loss: 5.370200157165527
Epoch: 0 Batch: 12000 Loss: 3.9275732040405273
Epoch: 0 Batch: 14000 Loss: 3.794468879699707
Epoch: 0 Batch: 16000 Loss: 3.8105435371398926
Epoch: 0 Batch: 18000 Loss: 3.3745758533477783
Epoch: 0 Batch: 20000 Loss: 4.577354431152344
Epoch: 0 Batch: 22000 Loss: 4.788670539855957
Epoch: 0 Batch: 24000 Loss: 3.61185884475708
Epoch: 0 Batch: 26000 Loss: 3.238779067993164
Epoch: 0 Batch: 28000 Loss: 4.005866050720215
Epoch: 0 Batch: 30000 Loss: 2.8864517211914062
Epoch: 0 Batch: 32000 Loss: 5.089885711669922
Epoch: 0 Batch: 34000 Loss: 4.502218246459961
Epoch: 0 Batch: 36000 Loss: 3.627053737640381
______________________________________
Epoch Loss: 146783.75069224834
handle older car in bag . yellow bicycle with a <unk> woman t

KeyboardInterrupt: 

In [ ]:
def predict_on_whole_val(model_encoder,model_decoder,encoder_encode_decode,decoder_itow,decoder_wtoi):
    model_encoder.eval()
    model_decoder.eval()
    accuracy_tuple_list = []  #[(jumbed_sent,unjumbled_sent,predicted_sent,hard,soft,word_count),...,]
    with torch.no_grad():
        for data_index in range(len(Xval_e)):
            if data_index % 50 == 0: print(data_index,end = ' ')
            Xe_b = torch.tensor([encoder_encode_decode.get_encoding(Xval_e[data_index])]).to(device)

            init_ht_for_encoder = model_encoder.init_hidden().to(device)
            op_from_encoder,ht,ht_for_decoder = model_encoder(Xe_b,init_ht_for_encoder)
            sos_word = torch.tensor([[decoder_wtoi["<sos>"]]]).to(device)
            op,ht,softmax_op = model_decoder(sos_word,ht_for_decoder,op_from_encoder)
            unjumbled_sentence = []
            for i in range(25):
                predicted_word = torch.argmax(op,axis=1).tolist()
                unjumbled_sentence.append(decoder_itow[predicted_word[0]])
                if predicted_word[0] == decoder_wtoi["<eos>"]: break
                op,ht,softmax_op = model_decoder(torch.tensor([predicted_word]).to(device),ht,op_from_encoder)
                
            hard_accuracy = 1 if " ".join(unjumbled_sentence) == Yval[data_index] else 0
            word_count = len(set(Yval[data_index].split()))
            soft_accuracy = len(set(unjumbled_sentence).intersection(set(Yval[data_index].split())))/word_count
            accuracy_tuple_list.append(
                (Xval_e[data_index],Yval[data_index]," ".join(unjumbled_sentence),hard_accuracy,soft_accuracy,word_count)
            )
    return accuracy_tuple_list

In [ ]:
accuracy_tuple_list = predict_on_whole_val(model_encoder,model_decoder,encoder_encode_decode,decoder_itow,decoder_wtoi)
df = pd.DataFrame(accuracy_tuple_list)
df.columns = ["jumbled_sent","unjumbled_sent","prediction","hard_accuracy","soft_accuracy","word_count"]
print(df.shape,df['hard_accuracy'].sum())
df.head(10)

In [ ]:
a = sum((df['soft_accuracy']*df['word_count']).tolist())
b = df['word_count'].sum()
a,b,a/b

In [ ]:
df[df['soft_accuracy']==1].shape, df[df['soft_accuracy']==1].shape[0]/df.shape[0]

In [ ]:
def predict_on_test(model_encoder_new,model_decoder_new,encoder_encode_decode,decoder_itow,decoder_wtoi):
    model_encoder_new.eval()
    model_decoder_new.eval()
    accuracy_tuple_list = []  #[(jumbed_sent,unjumbled_sent,predicted_sent,hard,soft,word_count),...,]
    with torch.no_grad():
        for data_index in range(len(Xtest_e)):
            if data_index % 50 == 0: print(data_index,end = ' ')
            Xe_b = torch.tensor([encoder_encode_decode.get_encoding(Xtest_e[data_index])]).to(device)
            init_ht_for_encoder = model_encoder_new.init_hidden().to(device)
            op_from_encoder,ht,ht_for_decoder = model_encoder_new(Xe_b,init_ht_for_encoder)
            sos_word = torch.tensor([[decoder_wtoi["<sos>"]]]).to(device)
            op,ht,softmax_op = model_decoder_new(sos_word,ht_for_decoder,op_from_encoder)
            unjumbled_sentence = []
            for i in range(25):
                predicted_word = torch.argmax(op,axis=1).tolist()
                unjumbled_sentence.append(decoder_itow[predicted_word[0]])
                if predicted_word[0] == decoder_wtoi["<eos>"]: break
                op,ht,softmax_op = model_decoder_new(torch.tensor([predicted_word]).to(device),ht,op_from_encoder)
            hard_accuracy = 1 if " ".join(unjumbled_sentence) == Ytest[data_index] else 0
            word_count = len(set(Ytest[data_index].split()))
            soft_accuracy = len(set(unjumbled_sentence).intersection(set(Ytest[data_index].split())))/word_count
            accuracy_tuple_list.append(
                (Xtest_e[data_index],Ytest[data_index]," ".join(unjumbled_sentence),hard_accuracy,soft_accuracy,word_count)
            )
    return accuracy_tuple_list

In [ ]:
Xtest_e = [
    "is eating . Nitish apple <eos>",
    "is city my favorite New York. <eos>",
    "a a and dog are man woods walking through the . <eos>"
]
Ytest = [
    "Nitish is eating apple . <eos>",
    "New York is my faorite city. <eos>",
    "a man and a dog are walking through the woods . <eos>"
]

In [ ]:
accuracy_tuple_list_test = predict_on_test(model_encoder,model_decoder,encoder_encode_decode,decoder_itow,decoder_wtoi)
df_test = pd.DataFrame(accuracy_tuple_list_test)
df_test.columns = ["jumbled_sent","unjumbled_sent","prediction","hard_accuracy","soft_accuracy","word_count"]
df_test.head(10)

## Save the models

#### Attention Model was not required to be saved explicitly as it is a part of the decoder model only.
#### Additionally the word_to_index and index_to_word dictionary and get_encoding funtion will be required for inference

In [ ]:
torch.save(model_encoder.state_dict(), "SavedModels/Jumble/encoder_model.pt")
torch.save(model_attention.state_dict(), "SavedModels/Jumble/attention_model.pt")
torch.save(model_decoder.state_dict(), "SavedModels/Jumble/decoder_model.pt")

# torch.save(model_encoder,"SavedModels/Jumble/encoder_model.pt")
# torch.save(model_attention,"SavedModels/Jumble/attention_model.pt")
# torch.save(model_decoder,"SavedModels/Jumble/decoder_model.pt")

## Load Saved Models

In [ ]:
device = torch.device("cuda:0")
hidden_size_encoder = 400
encoder_encode_decode = EncodeDecode(encoder_wtoi,encoder_itow,pad_token,unknown_token)
decoder_encode_decode = EncodeDecode(decoder_wtoi,decoder_itow,pad_token,unknown_token)
loaded_encoder_model = UnjumbleEncoderModel(
    vocab_size=len(encoder_wtoi),embedding_dim=300,num_lstm_layers=2,
    hidden_size=hidden_size_encoder,make_bidirectional=True,debug=False
).to(device)
loaded_encoder_model.load_state_dict(torch.load("SavedModels/Jumble/encoder_model.pt"))
loaded_encoder_model.eval()

if loaded_encoder_model.bidirectional: hidden_size_decoder = 2*hidden_size_encoder
loaded_attention_model = UnjumbleBahadnauAttention(hidden_size_decoder,debug=False).to(device)
loaded_attention_model.load_state_dict(torch.load("SavedModels/Jumble/attention_model.pt"))
loaded_attention_model.eval()

loaded_decoder_model = UnjumbleDecoderModel(model_attention = loaded_attention_model,
    vocab_size=len(encoder_wtoi),embedding_dim=300,num_lstm_layers=1,
    hidden_size=hidden_size_decoder,make_bidirectional=False,debug=False
).to(device)
loaded_decoder_model.load_state_dict(torch.load("SavedModels/Jumble/decoder_model.pt"))
loaded_decoder_model.eval()


# loaded_encoder_model = torch.load("SavedModels/Jumble/encoder_model.pt")
# loaded_decoder_model = torch.load("SavedModels/Jumble/decoder_model.pt")
# model_attention = torch.load("SavedModels/Jumble/attention_model.pt")

In [ ]:
accuracy_tuple_list_test = predict_on_test(loaded_encoder_model,loaded_decoder_model,encoder_encode_decode,decoder_itow,decoder_wtoi)
df_test = pd.DataFrame(accuracy_tuple_list_test)
df_test.columns = ["jumbled_sent","unjumbled_sent","prediction","hard_accuracy","soft_accuracy","word_count"]
df_test.head(10)